In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import collections
import copy

In [4]:
def plot_opt_decision_boundary(w, gamma, p):
    # given a linear model plot the decision boundary using the weights of the linear layer
    x = np.linspace(-3, 3, 100)
    y = -w[0]/w[1] * x - np.log(p/(1-p))/w[1]
    plt.plot(x, y, label="Optimal Decision Boundary")
    plt.legend()

In [5]:
def gen_data(n, d, w, gamma, p):
    ys = torch.distributions.Bernoulli(torch.tensor(p)).sample((n, 1))
    scaleDown = torch.tensor([1., 1.])
    zs = torch.randn(n, d)
    xs = (zs) / scaleDown[ys.long()] + gamma * (2*ys-1) * w
    return xs.float(), ys

In [6]:
def bayes_accuracy(xs, ys, w, gamma, p):
    temp = 1/(1+torch.exp(2*gamma*torch.matmul(xs, w)))
    preds = temp <= p
    return (preds == ys[:,0]).float().mean()

In [8]:
def train_on_source(xTrain, yTrain, xTest, yTest):
    '''
    Trains a logistic regression model on the source dataset and returns the model.
    '''
    model = nn.Sequential(nn.Linear(2, 128), nn.ReLU(), nn.Linear(128, 1))
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    losses = []
    for i in tqdm(range(10000)):
        optimizer.zero_grad()
        outputs = torch.sigmoid(model(xTrain))
        loss = F.binary_cross_entropy(outputs, yTrain)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    return model

In [9]:
def adapt(model, xTarget, epochs = 100000):
    '''
    Adapts the model to the target dataset and returns the adapted model.
    Current code uses Adam since the SGD was very slow.
    '''
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
    losses = []
    for i in tqdm(range(epochs)):
        optimizer.zero_grad()
        outputs = model(xTarget)

        probsPred = torch.sigmoid(outputs)
        labelPreds = (probsPred > 0.5).float().detach()
        loss = F.binary_cross_entropy(probsPred, labelPreds)
        losses.append(loss.item())

        # labelPreds = (outputs > 0).float().detach()
        # loss = torch.mean(torch.clamp(1 - outputs.t() * labelPreds, min=0))
        # loss = torch.mean(torch.exp(-outputs.t() * labelPreds))
        loss.backward()
        optimizer.step()
    return model

In [10]:
def evaluate(model, x, y):
    return((model(x) > 0) == y).float().mean().item()

In [11]:
n = 1000
d = 2
w = torch.ones(d)/np.sqrt(d)
gamma = 0.3
pSource = 0.5
pTarget = 0.9

In [12]:
xSourceTrain, ySourceTrain = gen_data(n, d, w, gamma, pSource)
xSourceTest, ySourceTest = gen_data(n, d, w, gamma, pSource)

xTargetTrain, yTargetTrain = gen_data(n, d, w, gamma, pTarget)
xTargetTest, yTargetTest = gen_data(n, d, w, gamma, pTarget)

In [13]:
bAccuSource = bayes_accuracy(xSourceTest, ySourceTest, w, gamma, pSource)
bAccuTarget = bayes_accuracy(xTargetTest, yTargetTest, w, gamma, pTarget)
print("Bayes accuracy on source is:", bAccuSource)
print("Bayes accuracy on target is:", bAccuTarget)

Bayes accuracy on source is: tensor(0.6270)
Bayes accuracy on target is: tensor(0.9100)


In [17]:
model = train_on_source(xSourceTrain, ySourceTrain, xSourceTest, ySourceTest)
print("Accuracy on source is:", evaluate(model, xSourceTest, ySourceTest))
print("Accuracy on target is:", evaluate(model, xTargetTest, yTargetTest))

model = adapt(model, xTargetTrain)
print("Accuracy on source is:", evaluate(model, xSourceTest, ySourceTest))
print("Accuracy on target is:", evaluate(model, xTargetTest, yTargetTest))

  0%|          | 0/10000 [00:00<?, ?it/s]

: 

: 

In [ ]:
accusBeforeAdapt = collections.defaultdict(list)
accusAfterAdapt = collections.defaultdict(list)
bayesAccus = collections.defaultdict(list)

data = collections.defaultdict(dict)



torch.manual_seed(0)
reps = 1
for _ in range(reps):
    xSourceTrain, ySourceTrain = gen_data(n, d, w, gamma, pSource)
    xSourceTest, ySourceTest = gen_data(n, d, w, gamma, pSource)
    model = train_on_source(xSourceTrain, ySourceTrain, xSourceTest, ySourceTest)
    data[_]['Source'] = [xSourceTrain, ySourceTrain, xSourceTest, ySourceTest]
    data[_]['Source_Model'] = copy.deepcopy(model)
    for i in range(10):
        p = 0.5 + i/20
        xTargetTrain, yTargetTrain = gen_data(n, d, w, gamma, p)
        xTargetTest, yTargetTest = gen_data(n, d, w, gamma, p)
        data[_][p] = [xTargetTrain, yTargetTrain, xTargetTest, yTargetTest]

        bAccuTarget = bayes_accuracy(xTargetTest, yTargetTest, w, gamma, p)
        bayesAccus[p].append(bAccuTarget)

        temp_model = copy.deepcopy(model)
        accusBeforeAdapt[p].append(evaluate(temp_model, xTargetTest, yTargetTest))
        temp_model = adapt(temp_model, xTargetTrain, epochs=250000)
        accusAfterAdapt[p].append(evaluate(temp_model, xTargetTest, yTargetTest))
        data[_][str(p)+'_Model'] = copy.deepcopy(temp_model)
        print(" Accu after adapt for", p, "is", accusAfterAdapt[p][-1])
# for p in accusAfterAdapt:
#     accusBeforeAdapt[p] /= reps
#     accusAfterAdapt[p] /= reps
#     bayesAccus[p] /= reps

In [ ]:
plt.scatter(list(bayesAccus.keys()), list(bayesAccus.values()), label="Bayes")
plt.plot(list(bayesAccus.keys()), list(bayesAccus.values()), label="Bayes")
plt.scatter(list(accusBeforeAdapt.keys()), list(accusBeforeAdapt.values()), label="Linear before adapt")
plt.plot(list(accusBeforeAdapt.keys()), list(accusBeforeAdapt.values()), label="Linear before adapt")
plt.scatter(list(accusAfterAdapt.keys()), list(accusAfterAdapt.values()), label="Linear after adapt")
plt.plot(list(accusAfterAdapt.keys()), list(accusAfterAdapt.values()), label="Linear after adapt")
plt.xlabel("probability of positive label")
plt.title("n={}, d={}, w={}, gamma={}, lr={}, epochs={}".format(n, d, w, gamma, 0.00001, 100000))
plt.ylabel("Accuracy")
plt.legend()

In [ ]:
numNegatives = []
for _ in data:
    for p in [0.5, 0.6, 0.7, 0.8, 0.9]:
        preds = data[_][p+'_Model'](data[_][p][0])
        numNegatives.append((preds < 0).float().mean().item())

        